In [ ]:
import earthaccess,csv
import xarray as xr
from xarray.backends.api import open_datatree
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
from scipy.ndimage import generic_filter
from scipy.ndimage import gaussian_filter
from scipy.interpolate import griddata
from scipy.ndimage import median_filter

from scipy.interpolate import LinearNDInterpolator

In [ ]:
%pip install haversine
import haversine as hs   
from haversine import Unit

In [ ]:
# Set default fontsizes for plots
fontsize = 18

plt.rc('font', size=fontsize)          # controls default text sizes
plt.rc('axes', titlesize=fontsize)     # fontsize of the axes title
plt.rc('axes', labelsize=fontsize)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=fontsize)    # fontsize of the tick labels
plt.rc('ytick', labelsize=fontsize)    # fontsize of the tick labels
plt.rc('legend', fontsize=fontsize)    # legend fontsize
plt.rc('figure', titlesize=fontsize)  # fontsize of the figure title

In [ ]:
def get_swot_data(latmin,latmax,lonmin,lonmax,tmin,tmax):
    """
    Searches for SWOT data within the bounding box and time. Note that this doesn't work for unsmoothed data. 
    
    latmin,latmax: latitude bounds, degrees N (south is negative); floats
    lonmin, lonmax: longitude bounds, degrees E (west is negative); floats
    tmin,tmax: temporal bounds; strings of form 'yyyy-mm-dd'
    """
    bbox = (lonmin, latmin, lonmax, latmax) # lonW, latS, lonE, latN
    results = earthaccess.search_data(
        short_name="SWOT_L2_LR_SSH_EXPERT_2.0",
        bounding_box=bbox,
        temporal=(tmin,tmax))
    
    print("Number of swaths: " + str(len(results))) # not daily files, so will likely be more than # of days
    paths = earthaccess.open(results) # is there a way to choose a subset of variables here?

    return paths

In [ ]:
latmin,latmax = 34,38
lonmin,lonmax = -73,-68
tmin,tmax = '2024-04-03','2024-08-05' #'2024-04-17','2024-08-04'

edward_paths = get_swot_data(latmin,latmax,lonmin,lonmax,tmin,tmax)

In [ ]:
date_requested = '20240508'

ind = 0 
for i in edward_paths:
    today = str(i).split('-')[-1].split('_')[-4][0:8]
    if date_requested == today:
        print(ind)
    ind += 1

In [ ]:
ds1 = xr.open_dataset(edward_paths[23])
ds2 = xr.open_dataset(edward_paths[24])

In [ ]:
def crop_dataset_by_lat_lon(ds,latmin,latmax,lonmin,lonmax):
    mask = (ds.latitude >= latmin) & (ds.latitude <= latmax) & (ds.longitude >= lonmin+360) & (ds.longitude <= lonmax+360)
    ds_masked = ds.where(mask, drop=True)
    return ds_masked

In [ ]:
ds1_masked = crop_dataset_by_lat_lon(ds1,latmin,latmax,lonmin,lonmax)
ds2_masked = crop_dataset_by_lat_lon(ds2,latmin,latmax,lonmin,lonmax)

In [ ]:
xv = np.arange(287, 293, 0.008)
yv = np.arange(33,39, 0.008)
grid_x, grid_y = np.meshgrid(xv, yv)

In [ ]:
# Quick velocities: need to change

def quick_velocities(ds, grid_x, grid_y):
    ssh=ds.ssha_karin_2+ds.height_cor_xover
    lats=ds.latitude
    lons=ds.longitude
    grid_ssh = griddata((lons.values.ravel(),lats.values.ravel()), ssh.values.ravel(), (grid_x, grid_y), method='linear')
    return grid_ssh

In [ ]:
grid_ssh1 = quick_velocities(ds1_masked, grid_x, grid_y)
grid_ssh2 = quick_velocities(ds2_masked, grid_x, grid_y)

In [ ]:
def compute_geos_current(ssh,lat):
    """
    ssh: (m) Make sure this is first corrected with height_cor_xover from L2 data! 
    lat: degrees N
    """
    
    omega = 7.2921159e-05  # angular velocity of the Earth [rad/s]
    fc = 2*omega*np.sin(lat*np.pi/180.)
        
    # avoid zero near equator, bound fc by min val as 1.e-8
    f_coriolis = np.sign(fc)*np.maximum(np.abs(fc), 1.e-8)
    
    dx,dy = 4000,4000 # m i changed it to 4000 to match res? need to double check
    gravity = 9.81

    dsdy,dsdx=np.array(np.gradient(ssh, dx, edge_order=1))
    vg = (gravity/np.array(f_coriolis))*dsdx
    ug = -(gravity/np.array(f_coriolis))*dsdy
    geos_current = np.sqrt(ug**2 + vg**2)
    
    return ug,vg,geos_current

In [ ]:
#Calculate
ug1,vg1,geos_current1 = compute_geos_current(grid_ssh1,grid_y)
ug2,vg2,geos_current2 = compute_geos_current(grid_ssh2,grid_y)

In [ ]:
UU = ug2
VV = vg2
xx=grid_x
yy=grid_y

speed = np.sqrt( (UU**2)+(VV**2) ) #is anyone doing EKE with SWOT?

#Convert to ualong
w=xx+1j*yy

ind=np.isfinite(w);
wr=np.copy(w)
w = w[ind]
cv = np.cov(np.stack([np.real(w), np.imag(w)]))
theta = 0.5 * np.arctan2(2 * cv[1, 0], (cv[0, 0] - cv[1, 1]))

# Find major and minor axis amplitudes
term1 = cv[0, 0] + cv[1, 1]
term2 = np.sqrt((cv[0, 0] - cv[1, 1])**2 + 4 * cv[1, 0]**2)
maj = np.sqrt(0.5 * (term1 + term2))
min_ = np.sqrt(0.5 * (term1 - term2))

# Rotate into principal ellipse orientation
wr[ind] = w * np.exp(-1j * theta) #-1j vs 1j changes direction of rotation

# Convert theta to degrees
theta_deg = theta * 180 / np.pi

xr = np.real(wr)
yr = np.imag(wr)

# Convert theta to radians
theta = np.deg2rad(theta_deg)

# Rotate vector (u + iv) by the angle theta
b = (UU + 1j * VV) * np.exp(-1j * theta)  # Angle relative to horizontal, rotate clockwise

# Extract u_along and u_across
u_along2 = np.real(b)
u_across2 = np.imag(b)

In [ ]:
UU = ug1
VV = vg1
xx=grid_x
yy=grid_y

speed = np.sqrt( (UU**2)+(VV**2) ) #is anyone doing EKE with SWOT?

#Convert to ualong
w=xx+1j*yy

ind=np.isfinite(w);
wr=np.copy(w)
w = w[ind]
cv = np.cov(np.stack([np.real(w), np.imag(w)]))
theta = 0.5 * np.arctan2(2 * cv[1, 0], (cv[0, 0] - cv[1, 1]))

# Find major and minor axis amplitudes
term1 = cv[0, 0] + cv[1, 1]
term2 = np.sqrt((cv[0, 0] - cv[1, 1])**2 + 4 * cv[1, 0]**2)
maj = np.sqrt(0.5 * (term1 + term2))
min_ = np.sqrt(0.5 * (term1 - term2))

theta=np.pi-theta #for ds_1

# Rotate into principal ellipse orientation
wr[ind] = w * np.exp(-1j * theta) #-1j vs 1j changes direction of rotation

# Convert theta to degrees
theta_deg = theta * 180 / np.pi

xr = np.real(wr)
yr = np.imag(wr)

# Convert theta to radians
theta = np.deg2rad(theta_deg)

# Rotate vector (u + iv) by the angle theta
b = (UU + 1j * VV) * np.exp(-1j * theta)  # Angle relative to horizontal, rotate clockwise

# Extract u_along and u_across
u_along = np.real(b)
u_across = np.imag(b)

In [ ]:
import utm

# Example latitude and longitude
latitude = grid_y
longitude = grid_x-360

# Convert to UTM
easting, northing, zone_number, zone_letter = utm.from_latlon(latitude, longitude)

In [ ]:
def compute_relative_vorticity(ug, vg, dx, dy):
    omega = 7.2921159e-05  # angular velocity of the Earth [rad/s]
    lat=36
    fc = 2*omega*np.sin(lat*np.pi/180.)
    # avoid zero near equator, bound fc by min val as 1.e-8
    f_coriolis = np.sign(fc)*np.maximum(np.abs(fc), 1.e-8)

    du_dy, du_dx = np.array(np.gradient(ug))
    du_dy = du_dy/dy
    du_dx=du_dx/dx
    
    dv_dy, dv_dx = np.array(np.gradient(vg))
    dv_dy=dv_dy/dy
    dv_dx=dv_dx/dx
        
    ksi = (dv_dx - du_dy)/f_coriolis
        
    return ksi

In [ ]:
ksi=compute_relative_vorticity(u_along, u_across, 1, 1)
ksi2=compute_relative_vorticity(u_along2, u_across2, 1, 1)

In [ ]:
fig,ax=plt.subplots(figsize=(7,4))
plot = ax.pcolormesh(xr, yr, u_across, shading='auto',cmap='coolwarm',vmin=-1,vmax=1)  # 'shading' option can adjust how cells are rendered
cbar = plt.colorbar(plot,ax=ax)
cbar.set_label('u_along', rotation=270, labelpad=15)

In [ ]:
fig,ax=plt.subplots(figsize=(7,4))
plot = ax.pcolormesh(grid_x, grid_y, ksi, shading='auto',cmap='coolwarm',vmin=-1,vmax=1)  # 'shading' option can adjust how cells are rendered
plot = ax.pcolormesh(grid_x, grid_y, ksi2, shading='auto',cmap='coolwarm',vmin=-1,vmax=1)  # 'shading' option can adjust how cells are rendered
cbar = plt.colorbar(plot,ax=ax)
cbar.set_label('vorticity', rotation=270, labelpad=15)

Convert u,v to u',v' in direction of swath to calculate geostrophic strain and geostrophic vorticity. Generate JPDFs. Do regions of trapping / mixing appear in distinct regions of this diagram?